In [34]:
import os
import cv2
import numpy as np
!pip install numpy opencv-python scikit-learn tensorflow

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report


In [6]:
from google.colab import drive

# Google Drive'a bağlanma
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
# Veri setinin bulunduğu klasörün yolu
data_folder = '/content/drive/My Drive/lung_image_sets'


In [15]:
DATADIR = '/content/drive/My Drive/lung_image_sets'

CATEGORIES = ["lung_adenocarcinoma","lung_benign tissue","lung_squamous cell carcinoma"]
classes = os.listdir(DATADIR)

In [16]:
IMG_SIZE = 100


In [17]:
all_data =[]

def create_all_data():
    for category in CATEGORIES:
        path=os.path.join(DATADIR, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                all_data.append([new_array,class_num])
            except Exception as e:
                pass
create_all_data()

In [21]:
print("Toplam Görüntü Sayısı: ", len(all_data))

Toplam Görüntü Sayısı:  15000


In [20]:
X = []
y = []

for categories, label in all_data:
  X.append(categories)
  y.append(label)

In [26]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state =42)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)


In [27]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

x_val = np.array(x_val)
y_val = np.array(y_val)

In [28]:
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(10800, 100, 100, 3)
(3000, 100, 100, 3)
(1200, 100, 100, 3)
(10800,)
(3000,)
(1200,)


In [29]:
import tensorflow as tf

train_yCl = tf.keras.utils.to_categorical(y_train, num_classes=3)
test_yCl = tf.keras.utils.to_categorical(y_test, num_classes=3)
valid_yCl = tf.keras.utils.to_categorical(y_val, num_classes=3)

In [31]:
model = Sequential([
    Flatten(input_shape=x_train[0].shape),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

Epoch 1/10
338/338 [==============================] - 23s 64ms/step - loss: 331.6242 - accuracy: 0.6668 - val_loss: 232.3783 - val_accuracy: 0.6333
Epoch 2/10
338/338 [==============================] - 22s 66ms/step - loss: 105.4770 - accuracy: 0.7207 - val_loss: 38.2535 - val_accuracy: 0.7683
Epoch 3/10
338/338 [==============================] - 21s 62ms/step - loss: 59.3993 - accuracy: 0.7325 - val_loss: 70.7228 - val_accuracy: 0.6742
Epoch 4/10
338/338 [==============================] - 20s 60ms/step - loss: 38.4473 - accuracy: 0.7347 - val_loss: 68.6285 - val_accuracy: 0.6350
Epoch 5/10
338/338 [==============================] - 21s 62ms/step - loss: 32.2169 - accuracy: 0.7343 - val_loss: 17.6132 - val_accuracy: 0.7292
Epoch 6/10
338/338 [==============================] - 21s 63ms/step - loss: 17.1380 - accuracy: 0.7469 - val_loss: 20.9688 - val_accuracy: 0.6383
Epoch 7/10
338/338 [==============================] - 21s 62ms/step - loss: 10.0789 - accuracy: 0.7435 - val_loss: 16.504

In [35]:
# Test verisi üzerinde tahmin yapma
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Accuracy
accuracy = accuracy_score(y_test, y_pred_classes)
print(f"Accuracy: {accuracy}")

# Precision
precision = precision_score(y_test, y_pred_classes, average='macro')
print(f"Precision: {precision}")

# Recall
recall = recall_score(y_test, y_pred_classes, average='macro')
print(f"Recall: {recall}")

# F1 Score
f1 = f1_score(y_test, y_pred_classes, average='macro')
print(f"F1 Score: {f1}")

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)
print(f"Confusion Matrix:\n{conf_matrix}")

# Classification Report
class_report = classification_report(y_test, y_pred_classes)
print(f"Classification Report:\n{class_report}")

94/94 [==============================] - 2s 23ms/step
Accuracy: 0.745
Precision: 0.7457423176410485
Recall: 0.7510289780904471
F1 Score: 0.7328249449366994
Confusion Matrix:
[[478 164 395]
 [ 15 953   2]
 [150  39 804]]
Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.46      0.57      1037
           1       0.82      0.98      0.90       970
           2       0.67      0.81      0.73       993

    accuracy                           0.74      3000
   macro avg       0.75      0.75      0.73      3000
weighted avg       0.75      0.74      0.73      3000

